In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import ViTForImageClassification
import pandas as pd
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# 配置参数
CFG = {
	'img_size': 224,
	'batch_size': 16,      # 根据8GB显存调整
	'num_epochs': 30,
	'lr': 3e-5,
	'num_workers': 0,      # Windows系统需要设置为0
	'device': 'cuda' if torch.cuda.is_available() else 'cpu',
	'num_classes': 881,
	'seed': 42
}

# 设置随机种子
torch.manual_seed(CFG['seed'])

# 数据预处理
train_transform = transforms.Compose([
	transforms.RandomResizedCrop(CFG['img_size']),
	transforms.RandomHorizontalFlip(),
	transforms.RandomRotation(15),
	transforms.ToTensor(),
	transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
	transforms.Resize(CFG['img_size']),
	transforms.CenterCrop(CFG['img_size']),
	transforms.ToTensor(),
	transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 自定义数据集类
# 修正后的MedicineDataset类
class MedicineDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)  # 确保索引正确
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_path = self.df.loc[idx, 'path']
        image = Image.open(img_path).convert('RGB')
        label = self.df.loc[idx, 'label']
        
        if self.transform:
            image = self.transform(image)
            
        # 确保标签为torch.int64类型
        return image, torch.as_tensor(label, dtype=torch.long)

# 加载数据
def load_data(csv_path, root_dir):
    # 改进的CSV读取方式
    df = pd.read_csv(csv_path, header=0)  # 使用第一行作为列头
    df = df.rename(columns={'ID': 'path', 'Label': 'label'})  # 规范列名
    
    # 数据类型转换
    convert_dict = {'path': str, 'label': int}
    df = df.astype(convert_dict)
    
    # 路径修正（根据实际文件结构可能需要调整）
    df['path'] = df['path'].apply(lambda x: os.path.join(root_dir, x))
    
    # 验证数据有效性
    print("正在验证数据完整性...")
    valid_samples = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        if os.path.exists(row['path']):
            valid_samples.append(idx)
        else:
            print(f"警告：缺失文件 {row['path']}")
    df = df.loc[valid_samples]
    
    # 过滤无效类别（每个类别至少需要2个样本）
    label_counts = df['label'].value_counts()
    valid_labels = label_counts[label_counts >= 2].index
    df = df[df['label'].isin(valid_labels)]
    
    print(f"有效数据量: {len(df)}")
    print(f"有效类别数: {df['label'].nunique()}")
    
    # 转换为0-based标签
    df['label'] = df['label'] - 1
    
    # 分层划分数据集
    train_df, val_df = train_test_split(
        df,
        test_size=0.2,
        stratify=df['label'],
        random_state=CFG['seed']
    )
    return train_df, val_df
# 创建模型
def create_model():
	model = ViTForImageClassification.from_pretrained(
		'google/vit-base-patch16-224-in21k',
		num_labels=CFG['num_classes'],
		ignore_mismatched_sizes=True
	)
	return model.to(CFG['device'])

# 训练函数
def train_model(model, train_loader, val_loader):
	optimizer = torch.optim.AdamW(model.parameters(), lr=CFG['lr'])
	criterion = nn.CrossEntropyLoss()
	
	best_acc = 0.0
	for epoch in range(CFG['num_epochs']):
		# 训练阶段
		model.train()
		train_loss = 0.0
		progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1} [Train]')
		for images, labels in progress_bar:
			images = images.to(CFG['device'])
			labels = labels.to(CFG['device'])
			
			optimizer.zero_grad()
			
			outputs = model(images)
			loss = criterion(outputs.logits, labels)
			
			loss.backward()
			optimizer.step()
			
			train_loss += loss.item() * images.size(0)
			progress_bar.set_postfix(loss=loss.item())
		
		# 验证阶段
		model.eval()
		val_loss = 0.0
		correct = 0
		total = 0
		with torch.no_grad():
			for images, labels in tqdm(val_loader, desc=f'Epoch {epoch+1} [Val]'):
				images = images.to(CFG['device'])
				labels = labels.to(CFG['device'])
				
				outputs = model(images)
				loss = criterion(outputs.logits, labels)
				
				val_loss += loss.item() * images.size(0)
				_, predicted = torch.max(outputs.logits, 1)
				total += labels.size(0)
				correct += (predicted == labels).sum().item()
		
		# 打印统计信息
		train_loss = train_loss / len(train_loader.dataset)
		val_loss = val_loss / len(val_loader.dataset)
		val_acc = 100 * correct / total
		print(f'Epoch {epoch+1}/{CFG["num_epochs"]}')
		print(f'Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%')
		
		# 保存最佳模型
		if val_acc > best_acc:
			best_acc = val_acc
			torch.save(model.state_dict(), 'best_vit_model.pth')
	
	print(f'Best Validation Accuracy: {best_acc:.2f}%')

# 生成预测结果
def generate_predictions(model, test_dir, transform):
	model.eval()
	predictions = []
	
	# 获取测试图片列表
	test_images = [os.path.join(test_dir, f) for f in os.listdir(test_dir) 
				  if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
	
	# 创建测试数据加载器
	class TestDataset(Dataset):
		def __init__(self, image_paths, transform=None):
			self.image_paths = image_paths
			self.transform = transform
			
		def __len__(self):
			return len(self.image_paths)
		
		def __getitem__(self, idx):
			image = Image.open(self.image_paths[idx]).convert('RGB')
			if self.transform:
				image = self.transform(image)
			return image, os.path.basename(self.image_paths[idx])
	
	test_dataset = TestDataset(test_images, transform=transform)
	test_loader = DataLoader(test_dataset, batch_size=CFG['batch_size'], shuffle=False)
	
	# 进行预测
	with torch.no_grad():
		for images, filenames in tqdm(test_loader, desc='Predicting'):
			images = images.to(CFG['device'])
			outputs = model(images)
			_, preds = torch.max(outputs.logits, 1)
			
			for fn, pred in zip(filenames, preds.cpu().numpy()):
				predictions.append((fn, pred + 1))  # 转换回1-based
	
	# 保存结果
	with open('submission.txt', 'w') as f:
		for fn, pred in predictions:
			f.write(f'{fn}\t{pred}\n')

if __name__ == '__main__':
	# 数据准备
	train_df, val_df = load_data('chinese-medicine-image/train_labels.csv', 'chinese-medicine-image')
	
	# 创建数据集
	train_dataset = MedicineDataset(train_df, train_transform)
	val_dataset = MedicineDataset(val_df, val_transform)
	# 创建数据加载器
	train_loader = DataLoader(train_dataset, batch_size=CFG['batch_size'], shuffle=True)
	val_loader = DataLoader(val_dataset, batch_size=CFG['batch_size'], shuffle=False)
	
	# 初始化模型
	model = create_model()
	
	# 开始训练
	train_model(model, train_loader, val_loader)
	
	# 加载最佳模型进行预测
	model.load_state_dict(torch.load('best_vit_model.pth'))
	
	# 生成提交文件（假设测试图片在test目录下）
	generate_predictions(model, 'chinese-medicine-image/test', val_transform)


正在验证数据完整性...


100%|██████████| 167017/167017 [00:10<00:00, 16399.91it/s]


有效数据量: 167017
有效类别数: 879


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1 [Train]:   0%|          | 5/8351 [00:01<46:15,  3.01it/s, loss=6.79]  c:\Users\wyz20\anaconda3\Lib\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Epoch 1 [Val]: 100%|██████████| 2088/2088 [13:00<00:00,  2.68it/s]


Epoch 1/30
Train Loss: 5.3754 | Val Loss: 3.7197 | Val Acc: 40.38%


...

Epoch 29 [Val]: 100%|██████████| 2088/2088 [13:39<00:00,  2.55it/s]


Epoch 29/30
Train Loss: 0.3424 | Val Loss: 2.0424 | Val Acc: 63.53%


Epoch 30 [Val]: 100%|██████████| 2088/2088 [13:39<00:00,  2.55it/s]


Epoch 30/30
Train Loss: 0.3361 | Val Loss: 2.0567 | Val Acc: 63.36%
Best Validation Accuracy: 64.76%


Predicting: 100%|██████████| 4694/4694 [12:22<00:00,  6.32it/s]
